# Segmented Spacetime — One‑Click Colab Runner

This notebook runs the **auto‑mode** evaluation for Segmented Spacetime (SSZ) with detailed terminal output,
paired comparison vs. GR/SR/GR×SR, equal‑count mass bins, and optional residuals + plots.

**No local clone is required.** The notebook fetches just the needed files directly (CSV + module).

In [ ]:
#@title Setup (installs minimal deps)
import sys, subprocess
pkgs = ['numpy', 'pandas', 'matplotlib']
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet'] + pkgs)
print('OK — deps ready')

In [ ]:
#@title Fetch data & module (auto; no repo clone)
import os, hashlib, time, urllib.request, pathlib, textwrap
from pathlib import Path

# You can change these raw URLs to your preferred branches/commits if needed:
RAW_CSV_URL = 'https://raw.githubusercontent.com/LinoCasu/Segmented-Spacetime-Mass-Projection-Unified-Results/main/real_data_full.csv'
RAW_MOD_URL = 'https://raw.githubusercontent.com/LinoCasu/Segmented-Spacetime-Mass-Projection-Unified-Results/main/segspace_all_in_one_extended.py'

work = Path('.')
csv_path = work / 'real_data_full.csv'
mod_path = work / 'segspace_all_in_one_extended.py'

def download(url, out):
    with urllib.request.urlopen(url) as r, open(out, 'wb') as f:
        f.write(r.read())

def sha256(p: Path):
    h = hashlib.sha256()
    with open(p, 'rb') as f:
        for chunk in iter(lambda: f.read(8192), b''):
            h.update(chunk)
    return h.hexdigest()

download(RAW_CSV_URL, csv_path)
download(RAW_MOD_URL, mod_path)

print('CSV:', csv_path.resolve())
print('SHA256:', sha256(csv_path))
print('MOD:', mod_path.resolve())
print('SHA256:', sha256(mod_path))

In [ ]:
#@title Define runner (v4: equal-count bins, optional raws/plots, pretty formatting)
import os, json, math, time, hashlib, textwrap
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

M_sun = 1.98847e30
BANNER = '='*88
SUB = '-'*88

def fmt_sig(x, sig=3):
    try:
        x = float(x)
    except:
        return 'n/a'
    if not np.isfinite(x):
        return 'n/a'
    ax = abs(x)
    if (ax != 0 and (ax >= 1e6 or ax < 1e-3)):
        return f"{x:.{sig}e}"
    s = f"{x:.{sig}g}"
    if 'e' not in s and '.' not in s:
        s = s + '.0'
    return s

def sha256(p: Path):
    h = hashlib.sha256()
    with open(p, 'rb') as f:
        for chunk in iter(lambda: f.read(8192), b''):
            h.update(chunk)
    return h.hexdigest()

def finite(x):
    try:
        return np.isfinite(float(x))
    except Exception:
        return False

def maybe_save_raws(raws, outdir, enable=False):
    if not enable: return None
    outdir.mkdir(parents=True, exist_ok=True)
    raw_path = outdir / 'raws_full.csv'
    pd.DataFrame(raws).to_csv(raw_path, index=False)
    return raw_path

def maybe_plots(raws, outdir, enable=False):
    if not enable: return []
    outdir.mkdir(parents=True, exist_ok=True)
    df = pd.DataFrame(raws)
    figs = []
    for key, label in [('abs_seg','SEG'), ('abs_grsr','GR×SR'), ('abs_gr','GR'), ('abs_sr','SR')]:
        if key in df and df[key].notna().any():
            plt.figure(figsize=(7,5))
            df[key].dropna().astype(float).plot(kind='hist', bins=30)
            plt.xlabel('|Δz|'); plt.ylabel('Count'); plt.title(f'Histogram of |Δz| — {label}')
            p = outdir / f'hist_abs_{label.replace("×","x")}.png'
            plt.tight_layout(); plt.savefig(p, dpi=300); plt.close(); figs.append(p)
    if {'abs_seg','abs_grsr'}.issubset(df.columns):
        s = df['abs_seg'].dropna().astype(float).sort_values(); g = df['abs_grsr'].dropna().astype(float).sort_values()
        if len(s)>0 and len(g)>0:
            ys = (np.arange(1, len(s)+1))/len(s); yg = (np.arange(1, len(g)+1))/len(g)
            plt.figure(figsize=(7,5))
            plt.plot(s.values, ys, label='SEG'); plt.plot(g.values, yg, label='GR×SR')
            plt.xlabel('|Δz|'); plt.ylabel('ECDF'); plt.title('ECDF — SEG vs GR×SR'); plt.legend()
            p = outdir / 'ecdf_abs_SEG_vs_GRSR.png'
            plt.tight_layout(); plt.savefig(p, dpi=300); plt.close(); figs.append(p)
    if {'abs_seg','abs_grsr'}.issubset(df.columns):
        s = df['abs_seg'].dropna().astype(float).values; g = df['abs_grsr'].dropna().astype(float).values
        if len(s)>0 and len(g)>0:
            plt.figure(figsize=(7,5))
            plt.boxplot([s,g], labels=['SEG','GR×SR']); plt.ylabel('|Δz|'); plt.title('SEG vs GR×SR — |Δz|')
            p = outdir / 'box_abs_SEG_vs_GRSR.png'
            plt.tight_layout(); plt.savefig(p, dpi=300); plt.close(); figs.append(p)
    return figs

def run(save_raws=False, plots=False):
    from pathlib import Path
    import runpy, math
    work = Path('.')
    csv_path = work / 'real_data_full.csv'
    mod_path = work / 'segspace_all_in_one_extended.py'
    ns = runpy.run_path(str(mod_path))
    load_csv = ns['load_csv']
    evaluate_redshift = ns['evaluate_redshift']
    z_gravitational = ns['z_gravitational']
    z_special_rel = ns['z_special_rel']
    z_combined = ns['z_combined']
    z_seg_pred = ns['z_seg_pred']
    A = float(ns['A']); B = float(ns['B']); ALPHA = float(ns['ALPHA'])

    print(BANNER); print(' SEGMENTED SPACETIME — AUTO RUN (NO ARGS)'); print(BANNER)
    print('Deterministic SSZ evaluation with phi/2 coupling and fixed Delta(M).\n')
    print(BANNER); print(' INPUTS & PROVENANCE (REPRODUCIBILITY)'); print(BANNER)
    print('CSV file     :', csv_path.resolve()); print('CSV sha256   :', sha256(csv_path))
    print('Module file  :', mod_path.resolve()); print('Module sha256:', sha256(mod_path))
    print('\n' + BANNER); print(' PHYSICAL CONSTANTS & MODEL CHOICES'); print(BANNER)
    print(f'Delta(M) parameters (fixed): A={A}, B={B}, ALPHA={ALPHA} (slow; dimensionless after norm).')
    print('PPN outer series: beta=gamma=1 (unchanged).')

    print(f"[ECHO] Loading CSV: {csv_path.resolve()}")
    rows = load_csv(csv_path)
    print(f"[ECHO] [OK] loaded rows: {len(rows)}")
    print(BANNER); print(' DATASET SUMMARY'); print(BANNER)
    print('Rows loaded              :', len(rows))
    print('Hybrid use of z (observed) else z_geom_hint. Deterministic predictions; no fit.\n')

    # Headline medians/sign-test
    res = evaluate_redshift(rows=rows, prefer_z=True, mode='hybrid',
                            dmA=A, dmB=B, dmAlpha=ALPHA, lo=None, hi=None,
                            drop_na=False, paired_stats=True, n_boot=0, bins=8,
                            do_plots=False, out_fig_dir=None, filter_complete_gr=True)
    med = res.get('med', {}); paired = res.get('paired', {})

    # Raw residuals
    raws = []
    for r in rows:
        z_obs = r.get('z'); z_geom = r.get('z_geom_hint')
        z_use = float(z_obs) if finite(z_obs) else (float(z_geom) if finite(z_geom) else None)
        Msun = r.get('M_solar'); Msun = float(Msun) if finite(Msun) else float('nan')
        Mkg = Msun * M_sun if finite(Msun) else float('nan')
        rem = r.get('r_emit_m'); rem = float(rem) if finite(rem) else float('nan')
        v_los = float(r.get('v_los_mps', 0.0)) if finite(r.get('v_los_mps', 0.0)) else 0.0
        v_tot = float(r.get('v_tot_mps', float('nan'))) if finite(r.get('v_tot_mps', float('nan'))) else float('nan')
        if not (finite(Mkg) and finite(rem) and rem > 0 and z_use is not None):
            continue
        zgr = z_gravitational(Mkg, rem)
        zsr = z_special_rel(v_tot, v_los)
        zgrsr = z_combined(zgr, zsr)
        lM = math.log10(Mkg) if finite(Mkg) and Mkg > 0 else math.log10(M_sun)
        zseg = z_seg_pred('hybrid', float(z_geom) if finite(z_geom) else None,
                          zgr, zsr, zgrsr, A, B, ALPHA, lM, None, None)
        def adiff(zm):
            try: return abs(float(z_use) - float(zm))
            except: return float('nan')
        raws.append({'case': r.get('case'), 'M_solar': Msun,
                     'abs_seg': adiff(zseg), 'abs_grsr': adiff(zgrsr),
                     'abs_gr': adiff(zgr), 'abs_sr': adiff(zsr)})

    # Equal-count bins by M_solar
    valid = [row for row in raws if finite(row['M_solar'])]
    valid.sort(key=lambda r: r['M_solar'])
    bins = []
    if len(valid) >= 8:
        parts = np.array_split(np.arange(len(valid)), 8)
        for i, idxs in enumerate(parts):
            sel = [valid[j] for j in idxs]
            lo = sel[0]['M_solar']; hi = sel[-1]['M_solar']
            def med_key(k):
                vals = [r[k] for r in sel if finite(r[k])];
                return float(pd.Series(vals).median()) if len(vals)>0 else float('nan')
            bins.append({'bin': i, 'lo': float(lo), 'hi': float(hi), 'count': len(sel),
                         'med_seg': med_key('abs_seg'), 'med_grsr': med_key('abs_grsr')})
    else:
        def med_key_all(k):
            vals = [r[k] for r in valid if finite(r[k])] ;
            return float(pd.Series(vals).median()) if len(vals)>0 else float('nan')
        bins = [{'bin': 0, 'lo': float('nan'), 'hi': float('nan'), 'count': len(valid),
                 'med_seg': med_key_all('abs_seg'), 'med_grsr': med_key_all('abs_grsr')}]

    # Print headline
    print(BANNER); print(' ACCURACY — MEDIAN ABSOLUTE REDSHIFT ERROR |Δz| (LOWER IS BETTER)'); print(BANNER)
    for key, name in [('seg','SSZ (phi/2 + DeltaM)'), ('grsr','GR×SR'), ('gr','GR'), ('sr','SR')]:
        if key in med and med[key] is not None:
            print(f"{name:<22}: {med[key]:.6g}")
    print()
    if 'paired' in res and res['paired']:
        p = res['paired']
        print(SUB); print(' PAIRED SIGN-TEST — SSZ vs GR×SR (per-row absolute errors)'); print(SUB)
        for k in ['N_pairs','N_Seg_better','share_Seg_better','binom_two_sided_p']:
            v = p.get(k)
            print(f"{k:<22}: {v}")
        print()
    print(SUB); print(' MASS-BINNED MEDIANS (SEG vs GR×SR) — with counts'); print(SUB)
    print(f"{'bin':>3} | {'lo→hi (M☉)':<24} | {'count':>5} | {'med_seg':>12} | {'med_grsr':>12}")
    print('-'*70)
    for b in bins:
        rng = f"[{fmt_sig(b['lo'])}…{fmt_sig(b['hi'])}]"
        print(f"{b['bin']:>3} | {rng:<24} | {b['count']:>5} | {float(b['med_seg']):>12.6g} | {float(b['med_grsr']):>12.6g}")
    print()

    # Save summary + optional artifacts
    repdir = Path('full_pipeline/reports'); figdir = Path('full_pipeline/figures')
    repdir.mkdir(parents=True, exist_ok=True)
    summary = {'med': med, 'paired': res.get('paired', {}), 'bins': bins,
               'raws': raws,
               'provenance': {
                   'csv': {'path': str(csv_path.resolve()), 'sha256': sha256(csv_path)},
                   'module': {'path': str(mod_path.resolve()), 'sha256': sha256(mod_path)},
                   'runner': 'colab-notebook-v4'
               }}
    out_json = repdir / 'summary_full_terminal_v4.json'
    with open(out_json, 'w', encoding='utf-8') as f:
        json.dump(summary, f, indent=2)

    raw_csv = maybe_save_raws(raws, repdir, enable=save_raws)
    figs = maybe_plots(raws, figdir, enable=plots)

    print(BANNER); print(' RUN COMPLETE'); print(BANNER)
    print('Summary JSON             :', out_json.resolve())
    if raw_csv: print('Raw residuals CSV        :', raw_csv.resolve())
    if figs:
        print('Figures:')
        for p in figs:
            print(' -', p.resolve())

print('Runner ready. Use the next cell to execute.')

In [ ]:
#@title Execute (auto-mode; toggle artifacts)
SAVE_RAWS = True  #@param {type:"boolean"}
PLOTS = True      #@param {type:"boolean"}
run(save_raws=SAVE_RAWS, plots=PLOTS)